In [33]:
import os

os.environ["AZURE_ACCOUNT_NAME"] = "ajdevreddit"
os.environ["AZURE_TABLE_ENDPOINT"] = "https://ajdevreddit.table.core.windows.net/"
os.environ["AZURE_QUEUE_ENDPOINT"] = "https://ajdevreddit.queue.core.windows.net/"
os.environ["AZURE_ACCOUNT_KEY"] = "+9066TCgdeVignRdy50G4qjmNoUJuibl9ERiTGzdV4fwkvgdV3aSVqgLwldgZxj/UpKLkkfXg+3k+AStjFI33Q=="
os.environ["AZURE_STORAGE_CONNECTION_STRING"] = "DefaultEndpointsProtocol=https;AccountName=ajdevreddit;AccountKey=+9066TCgdeVignRdy50G4qjmNoUJuibl9ERiTGzdV4fwkvgdV3aSVqgLwldgZxj/UpKLkkfXg+3k+AStjFI33Q==;BlobEndpoint=https://ajdevreddit.blob.core.windows.net/;QueueEndpoint=https://ajdevreddit.queue.core.windows.net/;TableEndpoint=https://ajdevreddit.table.core.windows.net/;FileEndpoint=https://ajdevreddit.file.core.windows.net/"
os.environ["AZURE_VISION_API_KEY"] = "2ee8459a379c4b73aef287d1cf1c4b73"
os.environ["AZURE_VISION_ENDPOINT"] = "https://aj-vision-ai.cognitiveservices.azure.com/"

In [34]:
import logging
import os

from azure.core.credentials import AzureNamedKeyCredential
from azure.core.paging import ItemPaged
from azure.data.tables import TableClient, TableEntity

logging.getLogger("azure.storage").setLevel(logging.WARNING)

from azure.data.tables import TableServiceClient


class TableAdapter(object):
	credential = AzureNamedKeyCredential(os.environ["AZURE_ACCOUNT_NAME"], os.environ["AZURE_ACCOUNT_KEY"])

	def __init__(self):
		self.service: TableServiceClient = TableServiceClient(endpoint=os.environ["AZURE_TABLE_ENDPOINT"], credential=self.credential)
		self.tables = self.service.list_tables()


	def get_table_service_client(self) -> TableServiceClient:
		return self.service

	def perform_odata_query(self, table_name: str, query: str) -> list[dict]:
		table_client: TableClient = self.get_table_client(table_name=table_name)
		entities: ItemPaged[TableEntity] = table_client.query_entities(query)
		return list(entities)


	def get_table_client(self, table_name: str) -> TableClient:
		service: TableServiceClient = self.get_table_service_client()
		return service.get_table_client(table_name=table_name)

	def upsert_entity_to_table(self, table_name: str, entity: dict):
		table_client: TableClient = self.get_table_client(table_name=table_name)
		table_client.upsert_entity(entity=entity)
		return

	def get_all_entities(self, table_name: str) -> list[dict]:
		table_client: TableClient = self.get_table_client(table_name=table_name)
		entities: ItemPaged[TableEntity] = table_client.list_entities()
		return list(entities)

	def get_table_client_instance(self, table_name: str) -> TableClient:
		service: TableServiceClient = self.get_table_service_client()
		return service.get_table_client(table_name=table_name)

	def get_entity(self, table_name: str, partition_key: str, row_key: str) -> TableEntity:
		table_client: TableClient = self.get_table_client(table_name=table_name)
		entity: TableEntity = table_client.get_entity(partition_key=partition_key, row_key=row_key)
		return entity

In [35]:
table_adapter: TableAdapter = TableAdapter()

client: TableClient = table_adapter.get_table_client(table_name="curationPrimary")

all_entities = list(client.query_entities("PartitionKey ne 'foo'", select=['PartitionKey', 'RowKey']))

In [36]:
extant_entity_list = [item['RowKey'] for item in all_entities]

In [37]:
import praw

reddit: praw.Reddit = praw.Reddit(client_id='5hVavL0PIRyM_1JSvqT6UQ', client_secret='BjD2kS3WNLnJc59RKY-JJUuc_Z9-JA',
								  user_agent='script:%(bot_name)s:v%(bot_version)s (by /u/%(bot_author)s)',
								  check_for_async=False)

In [38]:
from adlfs import AzureBlobFileSystem
from transformers import BlipProcessor, BlipForConditionalGeneration
import random
import hashlib
import torch
from PIL import Image

class BlipCaption:
	def __init__(self, device_name: str = "cuda"):
		self.processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
		self.device = torch.device(device_name)
		self.model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large",
																  torch_dtype=torch.float16).to(self.device)

	def caption_image(self, image_path: str) -> str:
		try:
			raw_image = Image.open(image_path).convert('RGB')
			inputs = self.processor(raw_image, return_tensors="pt").to(self.device, torch.float16)
			out = self.model.generate(**inputs)
			return self.processor.decode(out[0], skip_special_tokens=True, max_new_tokens=50)

		except Exception as e:
			print(e)
			return ""


class Functions(object):
	def get_name_for_image(self, x: object, file_list__) -> str:
		path = x['path']
		if path != "" and path in file_list__:
			return os.path.basename(path)
		else:
			return ""

	def set_exists(self, x: object, file_system__) -> bool:
		try:
			sub_reddit = x['subreddit']
			record_id = x['id']
			remote_path = x['path']
			return file_system__.exists(remote_path)
		except Exception as ex:
			return False

	def set_hash(self, x: object):
		sub_reddit = x['subreddit']
		record_id = x['id']
		temp_path = f"temp/image/{sub_reddit}/{record_id}.jpg"
		if os.path.exists(temp_path):
			return hashlib.md5(open(temp_path, 'rb').read()).hexdigest()
		else:
			return ""

	def add_source(self, x: object, source_list) -> str:
		sub_reddit = x['subreddit']
		for source in source_list:
			data_source = source['data']
			source_name = source['name']
			if sub_reddit in data_source:
				return source_name
		return ""

	def apply_caption(self, x: object, caption_routine: list[BlipCaption], _file_system: AzureBlobFileSystem) -> str:
		_exists = x['exists']
		if not _exists:
			return ""
		sub_reddit = x['subreddit']
		record_id = x['id']
		remote_path = x['path']
		os.makedirs(f"temp/image/{sub_reddit}/", exist_ok=True)
		temp_path = f"temp/image/{sub_reddit}/{record_id}.jpg"

		if os.path.exists(temp_path):
			try:
				result = random.choice(caption_routine).caption_image(temp_path)
				return result
			except Exception as ex:
				message = f"{x['id']}, {x['subreddit']}, Failure in apply_caption, {ex}"
				print(message)
				return ""
		else:
			try:
				_file_system.download(remote_path, temp_path)
				result = random.choice(caption_routine).caption_image(temp_path)
				return result
			except Exception as ex:
				message = f"{x['id']}, {x['subreddit']}, Failure in apply_caption, {ex}"
				print(message)
				return ""

	def fix_path(self, x: object, fl) -> str:
		current_path = x['path']
		exists_ = x['exists']
		if current_path in fl:
			return current_path
		else:
			image_id = x['id']
			if exists_:
				return f"data/image/{image_id}.jpg"
			else:
				return ""

In [39]:
from common.storage.azure_file_storage import AzureFileStorageAdapter
from tqdm import tqdm

# @title Image Acquisition Flow

display("==== Starting Image Acquisition ====")

tqdm.pandas(desc="Progress")

file_system = AzureFileStorageAdapter('data').get_file_storage()

functions: Functions = Functions()

'==== Starting Image Acquisition ===='

In [40]:
import json
subs = [
	"tightdresses",
	"Dresses",
	"SlitDresses",
	"CollaredDresses",
	"DressesPorn",
	"WomenInLongDresses",
	"TrueFMK",
	"DLAH",
	"SFWRedheads",
	"sfwpetite",
	"SFWNextDoorGirls",
	"realasians",
	"KoreanHotties",
	"prettyasiangirls",
	"AsianOfficeLady",
	"AsianInvasion",
	"AesPleasingAsianGirls",
	"sexygirls",
	"PrettyGirls",
	"gentlemanboners",
	"hotofficegirls",
	"Ifyouhadtopickone"
]

display("== Current Subs ==")
display(json.dumps(subs))

'== Current Subs =='

'["tightdresses", "Dresses", "SlitDresses", "CollaredDresses", "DressesPorn", "WomenInLongDresses", "TrueFMK", "DLAH", "SFWRedheads", "sfwpetite", "SFWNextDoorGirls", "realasians", "KoreanHotties", "prettyasiangirls", "AsianOfficeLady", "AsianInvasion", "AesPleasingAsianGirls", "sexygirls", "PrettyGirls", "gentlemanboners", "hotofficegirls", "Ifyouhadtopickone"]'

In [41]:
import pandas as pd

# @title Set Sources


sources = [
	{"name": "CityDiffusion", "data": ["CityPorn"]},
	{"name": "NatureDiffusion", "data": ["EarthPorn"]},
	{"name": "CosmicDiffusion", "data": ["spaceporn"]},
	{"name": "ITAPDiffusion", "data": ["itookapicture"]},
	{"name": "MemeDiffusion", "data": ["memes"]},
	{"name": "TTTDiffusion", "data": ["trippinthroughtime"]},
	{"name": "WallStreetDiffusion", "data": ["wallstreetbets"]},
	{"name": "SexyDiffusion", "data": ["selfies", "Amicute", "amihot", "AmIhotAF", "HotGirlNextDoor"]},
	{"name": "FatSquirrelDiffusion", "data": ["fatsquirrelhate"]},
	{"name": "CelebrityDiffusion", "data": ["celebrities"]},
	{"name": "OldLadyDiffusion", "data": ["oldladiesbakingpies"]},
	{"name": "SWFPetite", "data": ["sfwpetite"]},
	{"name": "SFWMilfs", "data": ["cougars_and_milfs_sfw"]},
	{"name": "RedHeadDiffusion", "data": ["SFWRedheads"]},
	{"name": "NextDoorGirlsDiffusion", "data": ["SFWNextDoorGirls"]},
	{"name": "SexyDressDiffusion", "data": ["SunDressesGoneWild", "ShinyDresses", "SlitDresses", "CollaredDresses", "DressesPorn","WomenInLongDresses", "Dresses", "tightdresses", "DLAH"]},
	{"name": "SexyAsianDiffusion", "data": ["realasians", "KoreanHotties", "prettyasiangirls", "AsianOfficeLady", "AsianInvasion", "AesPleasingAsianGirls"]},
	{"name": "MildlyPenisDiffusion", "data": ["mildlypenis"]},
	{"name": "PrettyGirlDiffusion", "data": ["sexygirls", "PrettyGirls", "gentlemanboners", "hotofficegirls","TrueFMK", "Ifyouhadtopickone"]},
	{"name": "CandleDiffusion", "data": ["bathandbodyworks"]}
]
sources_df = pd.DataFrame.from_records(sources)

display("== Sources To Apply ==")
display(sources_df.shape)
display(sources_df)

'== Sources To Apply =='

(20, 2)

,name,data
0,CityDiffusion,[CityPorn]
1,NatureDiffusion,[EarthPorn]
2,CosmicDiffusion,[spaceporn]
3,ITAPDiffusion,[itookapicture]
4,MemeDiffusion,[memes]
5,TTTDiffusion,[trippinthroughtime]
6,WallStreetDiffusion,[wallstreetbets]
7,SexyDiffusion,"[selfies, Amicute, amihot, AmIhotAF, HotGirlNe..."
8,FatSquirrelDiffusion,[fatsquirrelhate]
9,CelebrityDiffusion,[celebrities]


In [42]:
# @title Create Temp Dirs


for sub in tqdm(subs, total=len(subs), desc="Creating Temp Dir For Subs..."):
	temp_dir_path_ = f"temp/{sub}"
	if not os.path.exists(temp_dir_path_):
		os.makedirs(temp_dir_path_)
	os.makedirs(temp_dir_path_, exist_ok=True)

Creating Temp Dir For Subs...: 100%|██████████| 22/22 [00:00<00:00, 4265.46it/s]


In [43]:
# @title Locate And Cache Reddit Data
import pandas
from common.schemas.pyarrow_schema import schema
data = []
for sub in tqdm(subs, desc="Reading Temp Dir For Subs..."):
	df = pandas.read_parquet(f"temp/{sub}", schema=schema, engine='pyarrow')
	records = df.to_dict(orient='records')
	data.extend(records)

try:
	cache_data = pandas.DataFrame(data)
	cache_data.set_index('id', drop=False, inplace=True)
except KeyError:
	cache_data = pandas.DataFrame(data=[{'id': "foo"}])
	cache_data.set_index('id', drop=False, inplace=True)

for sub in subs:
	try:
		subreddit_stream_2 = list(reddit.subreddit(display_name=sub).hot(limit=100))
	except:
		subreddit_stream_2 = list()
	try:
		subreddit_stream_3 = list(reddit.subreddit(display_name=sub).new())
	except:
		subreddit_stream_3 = list()
	temp_dir_path = f"temp/{sub}"
	remote_temp_data = f"data/temp/{sub}"
	for time_filter in ['day', 'year', 'week', 'all', 'month']:
		try:
			subreddit_stream = reddit.subreddit(display_name=sub).top(limit=100, time_filter=time_filter)
			subreddit_stream = list(subreddit_stream)

			all_streams = subreddit_stream + subreddit_stream_2 + subreddit_stream_3
			all_streams = list(set(all_streams))
			p = tqdm(all_streams, total=len(all_streams), desc=f"Posts - {sub} - {time_filter}")
			for submission in p:
				if submission is None:
					continue
				else:
					if submission.id in extant_entity_list or submission.id in cache_data.index.values:
						continue
					else:
						try:
							if file_system.exists(f"{remote_temp_data}/{submission.id}.parquet"):
								file_system.download(f"{remote_temp_data}/{submission.id}.parquet",
													 f"{temp_dir_path}/{submission.id}.parquet")
								continue

							sub_id = submission.id
							author_name = 'Unknown'
							subreddit_name = sub
							sub_title = submission.title
							perma_link = submission.permalink
							sub_url = submission.url
							try:
								author_name = submission.author.name
							except Exception as e:
								author_name = 'Unknown'
								pass
							p = {
								'id': sub_id,
								'subreddit': subreddit_name,
								'author': author_name,
								'title': sub_title,
								'caption': '',
								'hash': '',
								'permalink': perma_link,
								'original_url': sub_url,
								'image_name': '',
								'path': '',
								'thumbnail_path': '',
								'exists': False,
								'curated': False,
								'Tags': ['']
							}
							continue
						except Exception as e:
							log = f"{submission.id}, {sub}, Error Writing Post, {e}"
							print(log)
							continue
		except Exception as e:
			log = f"{sub}, Error Getting Posts For SubReddit, {e}"
			print(log)
			continue

Posts - Ifyouhadtopickone - month: 100%|██████████| 205/205 [00:07<00:00, 27.94it/s]


In [44]:
import pandas
from common.schemas.pyarrow_schema import schema

# @title Read Cache And Stage Images

data = []
for sub in tqdm(subs, desc="Reading Temp Dir For Subs..."):
	df = pd.read_parquet(f"temp/{sub}", schema=schema, engine='pyarrow')
	records = df.to_dict(orient='records')
	data.extend(records)

temp_data = pandas.DataFrame(data)

for i, r in temp_data.iterrows():
	temp_data.loc[i, 'image_name'] = r.id + ".jpg"
	temp_data.loc[i, 'path'] = ""
	temp_data.loc[i, 'hash'] = ""
	temp_data.loc[i, 'caption'] = ""
	temp_data.loc[i, 'model'] = ""
	temp_data.loc[i, 'exists'] = False
	temp_data.loc[i, 'curated'] = False
	temp_data.loc[i, 'accept'] = False
	temp_data.loc[i, 'tags'] = ['']

temp_data.set_index('id', inplace=True, drop=False)

display("==== TEMP DATA ====")
display(temp_data.shape)
display(temp_data.head())

Reading Temp Dir For Subs...: 100%|██████████| 22/22 [00:02<00:00,  7.99it/s]


'==== TEMP DATA ===='

(4836, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
100pr72,100pr72,tightdresses,monkyCEO,Vacuum sealed hot pink,,,/r/tightdresses/comments/100pr72/vacuum_sealed...,https://i.imgur.com/claFblV.gifv,100pr72.jpg,,,False,False,False,[]
1034juw,1034juw,tightdresses,DiscreetNYC5,slip and slide 💙,,,/r/tightdresses/comments/1034juw/slip_and_slide/,https://gfycat.com/cluelesspointeddevilfish,1034juw.jpg,,,False,False,False,[]
107fr37,107fr37,tightdresses,Queenlivia00,Elegance is in me,,,/r/tightdresses/comments/107fr37/elegance_is_i...,https://i.imgur.com/T4JKrkz.gifv,107fr37.jpg,,,False,False,False,[]
10avknu,10avknu,tightdresses,bubblrisback,Making moves,,,/r/tightdresses/comments/10avknu/making_moves/,https://i.imgur.com/HabJw2C.gifv,10avknu.jpg,,,False,False,False,[]
10fm9ul,10fm9ul,tightdresses,Queenlivia00,Fit body,,,/r/tightdresses/comments/10fm9ul/fit_body/,https://i.imgur.com/HXjhy50.gifv,10fm9ul.jpg,,,False,False,False,[]


In [45]:
# @title Apply Filter For Images and Existance On Data

filtered = temp_data[~temp_data['id'].isin(extant_entity_list) & (temp_data['original_url'].str.endswith('.jpg') | temp_data['original_url'].str.endswith('.png'))]

filtered.dropna(axis=1, how='all')

filtered.reset_index(drop=True, inplace=True)

filtered.set_index('id', inplace=True, drop=False)

display("==== FILTERED DATA ====")

display(filtered.shape)

display(filtered.head())

'==== FILTERED DATA ===='

(35, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
139ygp2,139ygp2,tightdresses,TheFloweryDigress,19 y/o slut with a face of an Angel.... yeah I...,,,/r/tightdresses/comments/139ygp2/19_yo_slut_wi...,https://i.redgifs.com/i/carefulobesestarfish.jpg,139ygp2.jpg,,,False,False,False,[]
13bv2ug,13bv2ug,tightdresses,officialcategory,Undress me and make love to me tonight,,,/r/tightdresses/comments/13bv2ug/undress_me_an...,https://i.redgifs.com/i/wellmadevastelver.jpg,13bv2ug.jpg,,,False,False,False,[]
13c1i3n,13c1i3n,tightdresses,jackiemarielive,hope you don't mind my frizzy hair :),,,/r/tightdresses/comments/13c1i3n/hope_you_dont...,https://i.redgifs.com/i/annualfavoritegoldenma...,13c1i3n.jpg,,,False,False,False,[]
14302h8,14302h8,tightdresses,smaller_lemon,Your new sexytary,,,/r/tightdresses/comments/14302h8/your_new_sexy...,https://i.redgifs.com/i/delectablevainmullet.jpg,14302h8.jpg,,,False,False,False,[]
146k4nn,146k4nn,tightdresses,fairlyrustictaker,Is this tight enough for you?,,,/r/tightdresses/comments/146k4nn/is_this_tight...,https://i.redgifs.com/i/dramaticwarpednightjar...,146k4nn.jpg,,,False,False,False,[]


In [46]:
# @title Add Source To Model

with tqdm(total=len(filtered)) as pbar:
	for i, row in filtered.iterrows():
		d = functions.add_source(row, sources)
		filtered.loc[i, 'model'] = d
		pbar.update()

display("== Filtered Data With Model ==")
display(filtered.shape)
display(filtered.head())

100%|██████████| 35/35 [00:00<00:00, 3900.95it/s]


'== Filtered Data With Model =='

(35, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
139ygp2,139ygp2,tightdresses,TheFloweryDigress,19 y/o slut with a face of an Angel.... yeah I...,,,/r/tightdresses/comments/139ygp2/19_yo_slut_wi...,https://i.redgifs.com/i/carefulobesestarfish.jpg,139ygp2.jpg,,SexyDressDiffusion,False,False,False,[]
13bv2ug,13bv2ug,tightdresses,officialcategory,Undress me and make love to me tonight,,,/r/tightdresses/comments/13bv2ug/undress_me_an...,https://i.redgifs.com/i/wellmadevastelver.jpg,13bv2ug.jpg,,SexyDressDiffusion,False,False,False,[]
13c1i3n,13c1i3n,tightdresses,jackiemarielive,hope you don't mind my frizzy hair :),,,/r/tightdresses/comments/13c1i3n/hope_you_dont...,https://i.redgifs.com/i/annualfavoritegoldenma...,13c1i3n.jpg,,SexyDressDiffusion,False,False,False,[]
14302h8,14302h8,tightdresses,smaller_lemon,Your new sexytary,,,/r/tightdresses/comments/14302h8/your_new_sexy...,https://i.redgifs.com/i/delectablevainmullet.jpg,14302h8.jpg,,SexyDressDiffusion,False,False,False,[]
146k4nn,146k4nn,tightdresses,fairlyrustictaker,Is this tight enough for you?,,,/r/tightdresses/comments/146k4nn/is_this_tight...,https://i.redgifs.com/i/dramaticwarpednightjar...,146k4nn.jpg,,SexyDressDiffusion,False,False,False,[]


In [47]:
from retry import retry
import requests


@retry(Exception, tries=10, delay=3, jitter=2)
def make_trouble(url):
	return requests.get(url, stream=True)

def get_hash_from_path(in_path: str):
	if os.path.exists(in_path):
		with open(in_path, 'rb') as f_:
			content = f_.read()
			result = hashlib.md5(content).hexdigest()
			return result, content
	else:
		return ""


def fetch_image(x: object, file_list__, file_system__) -> str:
	try:
		url = x['original_url']
		subreddit = x['subreddit']
		image_id = x['id']
		os.makedirs(f"temp/image/{subreddit}", exist_ok=True)
		temp_path = f"temp/image/{subreddit}/{image_id}.jpg"
		out_path = f"data/image/{image_id}.jpg"
		if file_system__.exists(out_path):
			file_system__.download(out_path, temp_path)
		if os.path.exists(temp_path):
			md5, content = get_hash_from_path(temp_path)
			if md5 != "f17b01901c752c1bb04928131d1661af" or md5 != "d835884373f4d6c8f24742ceabe74946":
				if out_path in file_list__:
					file_system.download(out_path, temp_path)
					return out_path
				else:
					return out_path
			else:
				print("Invalid")
				return ""
		else:
			response = requests.get(url)
			md5 = hashlib.md5(response.content).hexdigest()
			if md5 != "f17b01901c752c1bb04928131d1661af" or md5 != "d835884373f4d6c8f24742ceabe74946":
				try:

					raw_image = Image.open(make_trouble(url).raw)
					if raw_image.mode in ("RGBA", "P"):
						raw_image = raw_image.convert("RGB")
					raw_image.save(temp_path)
					raw_image.close()
					if out_path in file_list__:
						return out_path
					else:
						file_system__.upload(temp_path, out_path)
						return out_path
				except Exception as ex:
					message = f"{x['id']}, {x['subreddit']}, Failure in fetch_image, {ex}, {url}"
					print(message)
					return ""
			else:
				return ""
	except Exception as ex:
		message = f"{x['id']}, {x['subreddit']}, Failure in fetch_image, {ex}"
		print(message)
		return ""

In [48]:
file_list = file_system.ls("data/image")
with tqdm(total=len(filtered)) as pbar:
	for i, row in filtered.iterrows():
		d = fetch_image(row, file_list, file_system)
		filtered.loc[i, 'path'] = d
		pbar.update()

display("== Filtered Data With Path ==")
display(filtered.shape)
display(filtered.head())

  3%|▎         | 1/35 [00:00<00:24,  1.38it/s]

139ygp2, tightdresses, Failure in fetch_image, cannot identify image file <_io.BytesIO object at 0x0000019FD603DEF0>, https://i.redgifs.com/i/carefulobesestarfish.jpg


  6%|▌         | 2/35 [00:01<00:18,  1.81it/s]

13bv2ug, tightdresses, Failure in fetch_image, cannot identify image file <_io.BytesIO object at 0x0000019FD609A400>, https://i.redgifs.com/i/wellmadevastelver.jpg


  9%|▊         | 3/35 [00:01<00:17,  1.84it/s]

...

100%|██████████| 35/35 [00:25<00:00,  1.39it/s]

13n119j, hotofficegirls, Failure in fetch_image, cannot identify image file <_io.BytesIO object at 0x0000019FD603CD60>, https://i.redgifs.com/i/colorlessdrearyxanclomys.jpg


'== Filtered Data With Path =='

(35, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
139ygp2,139ygp2,tightdresses,TheFloweryDigress,19 y/o slut with a face of an Angel.... yeah I...,,,/r/tightdresses/comments/139ygp2/19_yo_slut_wi...,https://i.redgifs.com/i/carefulobesestarfish.jpg,139ygp2.jpg,,SexyDressDiffusion,False,False,False,[]
13bv2ug,13bv2ug,tightdresses,officialcategory,Undress me and make love to me tonight,,,/r/tightdresses/comments/13bv2ug/undress_me_an...,https://i.redgifs.com/i/wellmadevastelver.jpg,13bv2ug.jpg,,SexyDressDiffusion,False,False,False,[]
13c1i3n,13c1i3n,tightdresses,jackiemarielive,hope you don't mind my frizzy hair :),,,/r/tightdresses/comments/13c1i3n/hope_you_dont...,https://i.redgifs.com/i/annualfavoritegoldenma...,13c1i3n.jpg,,SexyDressDiffusion,False,False,False,[]
14302h8,14302h8,tightdresses,smaller_lemon,Your new sexytary,,,/r/tightdresses/comments/14302h8/your_new_sexy...,https://i.redgifs.com/i/delectablevainmullet.jpg,14302h8.jpg,,SexyDressDiffusion,False,False,False,[]
146k4nn,146k4nn,tightdresses,fairlyrustictaker,Is this tight enough for you?,,,/r/tightdresses/comments/146k4nn/is_this_tight...,https://i.redgifs.com/i/dramaticwarpednightjar...,146k4nn.jpg,,SexyDressDiffusion,False,False,False,[]


In [49]:
def set_exists(x: object, file_system__) -> bool:
	try:
		sub_reddit = x['subreddit']
		record_id = x['id']
		remote_path = x['path']
		temp_path = f"temp/image/{sub_reddit}/{record_id}.jpg"
		if os.path.exists(temp_path) and file_system__.exists(remote_path):
			print("found in both local and remote")
			return True
		if os.path.exists(temp_path) and not file_system__.exists(remote_path):
			file_system__.upload(temp_path, remote_path)
			print("Only found in local, downloading to remote")
			return True
		if not os.path.exists(temp_path) and file_system__.exists(remote_path):
			file_system__.download(remote_path, temp_path)
			print("Only found in remote, downloading to local")
			return True
		else:
			print("Not found")
			return False
	except Exception as ex:
		print(ex)
		return False

In [50]:
# @title Apply Exist

for i, row in filtered.iterrows():
	d = set_exists(row, file_system)
	filtered.loc[i, 'exists'] = d

display("== Filtered Data With Exists ==")
display(filtered.shape)
display(filtered.head())

Please specify a container name and blob name.
Please specify a container name and blob name.
Please specify a container name and blob name.
Please specify a container name and blob name.
Please specify a container name and blob name.
Please specify a container name and blob name.
Please specify a container name and blob name.
Please specify a container name and blob name.
Please specify a container name and blob name.
found in both local and remote
found in both local and remote
found in both local and remote
found in both local and remote
found in both local and remote
found in both local and remote
found in both local and remote
found in both local and remote
found in both local and remote
found in both local and remote
found in both local and remote
found in both local and remote
found in both local and remote
found in both local and remote
found in both local and remote
found in both local and remote
found in both local and remote
found in both local and remote
found in both local

'== Filtered Data With Exists =='

(35, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
139ygp2,139ygp2,tightdresses,TheFloweryDigress,19 y/o slut with a face of an Angel.... yeah I...,,,/r/tightdresses/comments/139ygp2/19_yo_slut_wi...,https://i.redgifs.com/i/carefulobesestarfish.jpg,139ygp2.jpg,,SexyDressDiffusion,False,False,False,[]
13bv2ug,13bv2ug,tightdresses,officialcategory,Undress me and make love to me tonight,,,/r/tightdresses/comments/13bv2ug/undress_me_an...,https://i.redgifs.com/i/wellmadevastelver.jpg,13bv2ug.jpg,,SexyDressDiffusion,False,False,False,[]
13c1i3n,13c1i3n,tightdresses,jackiemarielive,hope you don't mind my frizzy hair :),,,/r/tightdresses/comments/13c1i3n/hope_you_dont...,https://i.redgifs.com/i/annualfavoritegoldenma...,13c1i3n.jpg,,SexyDressDiffusion,False,False,False,[]
14302h8,14302h8,tightdresses,smaller_lemon,Your new sexytary,,,/r/tightdresses/comments/14302h8/your_new_sexy...,https://i.redgifs.com/i/delectablevainmullet.jpg,14302h8.jpg,,SexyDressDiffusion,False,False,False,[]
146k4nn,146k4nn,tightdresses,fairlyrustictaker,Is this tight enough for you?,,,/r/tightdresses/comments/146k4nn/is_this_tight...,https://i.redgifs.com/i/dramaticwarpednightjar...,146k4nn.jpg,,SexyDressDiffusion,False,False,False,[]


In [51]:
# @title Apply Hash

for i, row in filtered.iterrows():
	d = functions.set_hash(row)
	filtered.loc[i, 'hash'] = d
display(filtered.shape)
display(filtered)

(35, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
139ygp2,139ygp2,tightdresses,TheFloweryDigress,19 y/o slut with a face of an Angel.... yeah I...,,,/r/tightdresses/comments/139ygp2/19_yo_slut_wi...,https://i.redgifs.com/i/carefulobesestarfish.jpg,139ygp2.jpg,,SexyDressDiffusion,False,False,False,[]
13bv2ug,13bv2ug,tightdresses,officialcategory,Undress me and make love to me tonight,,,/r/tightdresses/comments/13bv2ug/undress_me_an...,https://i.redgifs.com/i/wellmadevastelver.jpg,13bv2ug.jpg,,SexyDressDiffusion,False,False,False,[]
13c1i3n,13c1i3n,tightdresses,jackiemarielive,hope you don't mind my frizzy hair :),,,/r/tightdresses/comments/13c1i3n/hope_you_dont...,https://i.redgifs.com/i/annualfavoritegoldenma...,13c1i3n.jpg,,SexyDressDiffusion,False,False,False,[]
14302h8,14302h8,tightdresses,smaller_lemon,Your new sexytary,,,/r/tightdresses/comments/14302h8/your_new_sexy...,https://i.redgifs.com/i/delectablevainmullet.jpg,14302h8.jpg,,SexyDressDiffusion,False,False,False,[]
146k4nn,146k4nn,tightdresses,fairlyrustictaker,Is this tight enough for you?,,,/r/tightdresses/comments/146k4nn/is_this_tight...,https://i.redgifs.com/i/dramaticwarpednightjar...,146k4nn.jpg,,SexyDressDiffusion,False,False,False,[]
146zfz5,146zfz5,tightdresses,smaller_lemon,Tight dresses and sex on the beach...drink,,,/r/tightdresses/comments/146zfz5/tight_dresses...,https://i.redgifs.com/i/heavynewsugarglider.jpg,146zfz5.jpg,,SexyDressDiffusion,False,False,False,[]
ai2fux,ai2fux,DressesPorn,pizzaisavegetabIe,"Lupita Nyong'o, 2014 Oscars",,,/r/DressesPorn/comments/ai2fux/lupita_nyongo_2...,http://www.nicolegibbonsstyle.com/wp-content/u...,ai2fux.jpg,,SexyDressDiffusion,False,False,False,[]
13c3bc5,13c3bc5,SFWRedheads,Redhotwife88,Redhead Wife in my Little Black Dress! [F],,,/r/SFWRedheads/comments/13c3bc5/redhead_wife_i...,https://i.redgifs.com/i/admiredoutstandingpape...,13c3bc5.jpg,,RedHeadDiffusion,False,False,False,[]
13jm7mh,13jm7mh,SFWRedheads,Redhotwife88,Do I look Cute for my Anniversary Dinner? [F],,,/r/SFWRedheads/comments/13jm7mh/do_i_look_cute...,https://i.redgifs.com/i/tensedarkmagentabedlin...,13jm7mh.jpg,,RedHeadDiffusion,False,False,False,[]


In [52]:
from datetime import datetime

# @title Filter On Exists and path existance

ts = datetime.today().timestamp()

exists = filtered.loc[(filtered['exists'] == True) & (filtered['path'] != "")]
exists.dropna(axis=1, how='all')
exists.reset_index(drop=True, inplace=True)

display(exists.shape)
display(exists.head())

ready_to_caption = exists.copy()

(24, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
0,11qc33b,gentlemanboners,Halbkornbrot,Emma Watson,,5714acd1521779d9680709f8d53befd3,/r/gentlemanboners/comments/11qc33b/emma_watson/,https://i.redd.it/rp0kgb93uina1.png,11qc33b.jpg,data/image/11qc33b.jpg,PrettyGirlDiffusion,True,False,False,[]
1,11ssj7c,gentlemanboners,SalisburyShrimp,Betty Gilpin,,387242123419666b40d370245f6d5415,/r/gentlemanboners/comments/11ssj7c/betty_gilpin/,https://i.redd.it/gv7iv4cuf3oa1.png,11ssj7c.jpg,data/image/11ssj7c.jpg,PrettyGirlDiffusion,True,False,False,[]
2,13osh3e,gentlemanboners,throwaway08190924,Alexandra Daddario (Oct. 2022),,a260fad067cd7a9ba23496e025de3fe4,/r/gentlemanboners/comments/13osh3e/alexandra_...,https://i.redd.it/rqpykhp96c1b1.png,13osh3e.jpg,data/image/13osh3e.jpg,PrettyGirlDiffusion,True,False,False,[]
3,13ry9kc,gentlemanboners,onesexymofo1,Natalie Portman,,01f6aa5f3c897f9eb925ab4d10619d41,/r/gentlemanboners/comments/13ry9kc/natalie_po...,https://i.imgur.com/qE1wEHz.jpg,13ry9kc.jpg,data/image/13ry9kc.jpg,PrettyGirlDiffusion,True,False,False,[]
4,13tua2l,gentlemanboners,chuboi991,Hannah Waddingham,,56d6d1d2f06aaccbb82d71f5456eec9a,/r/gentlemanboners/comments/13tua2l/hannah_wad...,https://i.redd.it/pa39y7hogk2b1.png,13tua2l.jpg,data/image/13tua2l.jpg,PrettyGirlDiffusion,True,False,False,[]


In [53]:
# @title Import BLIP


import torch

caption_0 = BlipCaption("cuda")

In [54]:
# @title Filter on bad data


check = ready_to_caption.loc[(ready_to_caption['caption'] == "") & (ready_to_caption['caption'].notnull()) & (ready_to_caption['exists'] == True) & (~ready_to_caption['id'].isin(extant_entity_list))]

ready_to_caption = check.copy()

display("== Filtered On Existing Caption ==")
display(ready_to_caption.shape)
display(ready_to_caption.head())

'== Filtered On Existing Caption =='

(24, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
0,11qc33b,gentlemanboners,Halbkornbrot,Emma Watson,,5714acd1521779d9680709f8d53befd3,/r/gentlemanboners/comments/11qc33b/emma_watson/,https://i.redd.it/rp0kgb93uina1.png,11qc33b.jpg,data/image/11qc33b.jpg,PrettyGirlDiffusion,True,False,False,[]
1,11ssj7c,gentlemanboners,SalisburyShrimp,Betty Gilpin,,387242123419666b40d370245f6d5415,/r/gentlemanboners/comments/11ssj7c/betty_gilpin/,https://i.redd.it/gv7iv4cuf3oa1.png,11ssj7c.jpg,data/image/11ssj7c.jpg,PrettyGirlDiffusion,True,False,False,[]
2,13osh3e,gentlemanboners,throwaway08190924,Alexandra Daddario (Oct. 2022),,a260fad067cd7a9ba23496e025de3fe4,/r/gentlemanboners/comments/13osh3e/alexandra_...,https://i.redd.it/rqpykhp96c1b1.png,13osh3e.jpg,data/image/13osh3e.jpg,PrettyGirlDiffusion,True,False,False,[]
3,13ry9kc,gentlemanboners,onesexymofo1,Natalie Portman,,01f6aa5f3c897f9eb925ab4d10619d41,/r/gentlemanboners/comments/13ry9kc/natalie_po...,https://i.imgur.com/qE1wEHz.jpg,13ry9kc.jpg,data/image/13ry9kc.jpg,PrettyGirlDiffusion,True,False,False,[]
4,13tua2l,gentlemanboners,chuboi991,Hannah Waddingham,,56d6d1d2f06aaccbb82d71f5456eec9a,/r/gentlemanboners/comments/13tua2l/hannah_wad...,https://i.redd.it/pa39y7hogk2b1.png,13tua2l.jpg,data/image/13tua2l.jpg,PrettyGirlDiffusion,True,False,False,[]


In [55]:
# @title Apply Captions

with tqdm(desc="progress") as pbar:
	for i, row in ready_to_caption.iterrows():
		ready_to_caption.loc[i, 'caption'] = functions.apply_caption(row, [caption_0, caption_0], file_system)
		pbar.update()

display("== Data With Captions ==")
display(ready_to_caption.shape)
display(ready_to_caption.head())

progress: 0it [00:00, ?it/s]D:\code\repos\simple-collection\venv\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
progress: 24it [00:34,  1.45s/it]


'== Data With Captions =='

(24, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
0,11qc33b,gentlemanboners,Halbkornbrot,Emma Watson,woman in a gray dress standing next to a dress...,5714acd1521779d9680709f8d53befd3,/r/gentlemanboners/comments/11qc33b/emma_watson/,https://i.redd.it/rp0kgb93uina1.png,11qc33b.jpg,data/image/11qc33b.jpg,PrettyGirlDiffusion,True,False,False,[]
1,11ssj7c,gentlemanboners,SalisburyShrimp,Betty Gilpin,a close up of a woman in a purple dress posing...,387242123419666b40d370245f6d5415,/r/gentlemanboners/comments/11ssj7c/betty_gilpin/,https://i.redd.it/gv7iv4cuf3oa1.png,11ssj7c.jpg,data/image/11ssj7c.jpg,PrettyGirlDiffusion,True,False,False,[]
2,13osh3e,gentlemanboners,throwaway08190924,Alexandra Daddario (Oct. 2022),a close up of a woman in a white and black dress,a260fad067cd7a9ba23496e025de3fe4,/r/gentlemanboners/comments/13osh3e/alexandra_...,https://i.redd.it/rqpykhp96c1b1.png,13osh3e.jpg,data/image/13osh3e.jpg,PrettyGirlDiffusion,True,False,False,[]
3,13ry9kc,gentlemanboners,onesexymofo1,Natalie Portman,araffed - up of a woman in a red dress and sun...,01f6aa5f3c897f9eb925ab4d10619d41,/r/gentlemanboners/comments/13ry9kc/natalie_po...,https://i.imgur.com/qE1wEHz.jpg,13ry9kc.jpg,data/image/13ry9kc.jpg,PrettyGirlDiffusion,True,False,False,[]
4,13tua2l,gentlemanboners,chuboi991,Hannah Waddingham,a pregnant woman in a blue dress standing in a...,56d6d1d2f06aaccbb82d71f5456eec9a,/r/gentlemanboners/comments/13tua2l/hannah_wad...,https://i.redd.it/pa39y7hogk2b1.png,13tua2l.jpg,data/image/13tua2l.jpg,PrettyGirlDiffusion,True,False,False,[]


In [56]:
# @title Drop Missing Captions

dropped = ready_to_caption.dropna(axis=1, how='all')
dropped.reset_index(drop=True, inplace=True)
dropped.set_index("id", inplace=True, drop=False)

post_caption_drop = dropped.copy()

display("== Dropped And Reset Data After Captioning ==")
display(post_caption_drop.shape)
display(post_caption_drop.head())

'== Dropped And Reset Data After Captioning =='

(24, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
11qc33b,11qc33b,gentlemanboners,Halbkornbrot,Emma Watson,woman in a gray dress standing next to a dress...,5714acd1521779d9680709f8d53befd3,/r/gentlemanboners/comments/11qc33b/emma_watson/,https://i.redd.it/rp0kgb93uina1.png,11qc33b.jpg,data/image/11qc33b.jpg,PrettyGirlDiffusion,True,False,False,[]
11ssj7c,11ssj7c,gentlemanboners,SalisburyShrimp,Betty Gilpin,a close up of a woman in a purple dress posing...,387242123419666b40d370245f6d5415,/r/gentlemanboners/comments/11ssj7c/betty_gilpin/,https://i.redd.it/gv7iv4cuf3oa1.png,11ssj7c.jpg,data/image/11ssj7c.jpg,PrettyGirlDiffusion,True,False,False,[]
13osh3e,13osh3e,gentlemanboners,throwaway08190924,Alexandra Daddario (Oct. 2022),a close up of a woman in a white and black dress,a260fad067cd7a9ba23496e025de3fe4,/r/gentlemanboners/comments/13osh3e/alexandra_...,https://i.redd.it/rqpykhp96c1b1.png,13osh3e.jpg,data/image/13osh3e.jpg,PrettyGirlDiffusion,True,False,False,[]
13ry9kc,13ry9kc,gentlemanboners,onesexymofo1,Natalie Portman,araffed - up of a woman in a red dress and sun...,01f6aa5f3c897f9eb925ab4d10619d41,/r/gentlemanboners/comments/13ry9kc/natalie_po...,https://i.imgur.com/qE1wEHz.jpg,13ry9kc.jpg,data/image/13ry9kc.jpg,PrettyGirlDiffusion,True,False,False,[]
13tua2l,13tua2l,gentlemanboners,chuboi991,Hannah Waddingham,a pregnant woman in a blue dress standing in a...,56d6d1d2f06aaccbb82d71f5456eec9a,/r/gentlemanboners/comments/13tua2l/hannah_wad...,https://i.redd.it/pa39y7hogk2b1.png,13tua2l.jpg,data/image/13tua2l.jpg,PrettyGirlDiffusion,True,False,False,[]


In [58]:
# @title Ensure Valid

bar = post_caption_drop.index.values

display(f"Extant Contains New: {extant_entity_list in list(bar)} - {list(bar) in extant_entity_list}")

'Extant Contains New: False - False'

In [62]:
records = post_caption_drop.to_dict(orient='records')
for record in records:
	record['tags'] = json.dumps(record['tags'])
	record['PartitionKey'] = record['subreddit']
	record['RowKey'] = record['id']
	client.upsert_entity(record)